# Exploration et Machine Learning - Données Olympiques 🏆

Ce notebook présente une analyse complète des données olympiques avec :
- 📊 **Exploration des données** depuis Azure MySQL
- 🧹 **Nettoyage et préparation** des données
- 📈 **Visualisations interactives** avec Plotly
- 🤖 **Modèles de Machine Learning** avec Scikit-learn
  - Classification avec Random Forest
  - Clustering avec KMeans

## Structure des données
- **Athletes** : 75,904 athlètes olympiques
- **Hosts** : 53 pays/villes hôtes
- **Medals** : 17,011 médailles 
- **Results** : 113,632 résultats de compétitions

In [2]:
# Imports des bibliothèques pour l'exploration et le Machine Learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

# Bibliothèques Machine Learning
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score

# Base de données
import mysql.connector
from mysql.connector import Error

# Configuration d'affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Validation des imports
print("✅ Bibliothèques importées avec succès")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"📈 Plotly: {plotly.__version__}")
print(f"🤖 Scikit-learn disponible")
print(f"🗃️ MySQL Connector disponible")
print(f"🎨 Matplotlib et Seaborn configurés")

✅ Bibliothèques importées avec succès
📊 Pandas: 2.3.3
🔢 NumPy: 2.3.4
📈 Plotly: 6.3.1
🤖 Scikit-learn disponible
🗃️ MySQL Connector disponible
🎨 Matplotlib et Seaborn configurés


## 1. Connexion à la base de données et chargement des données

In [3]:
# Configuration de connexion Azure MySQL
def create_connection():
    """Crée une connexion à la base MySQL Azure"""
    try:
        config = {
            'host': 'olympics-m.mysql.database.azure.com',
            'user': 'azure7',
            'password': 'Gnarok246272',
            'database': 'olympics',
            'port': 3306,
            'ssl_disabled': False
        }
        conn = mysql.connector.connect(**config)
        print("✅ Connexion à Azure MySQL réussie")
        return conn
    except mysql.connector.Error as e:
        print(f"❌ Erreur de connexion: {e}")
        return None

# Chargement des données depuis la base
def load_olympic_data():
    """Charge toutes les données olympiques dans des DataFrames"""
    conn = create_connection()
    if not conn:
        return None, None, None, None
    
    try:
        # Chargement des tables
        print("📊 Chargement des données...")
        
        athletes_df = pd.read_sql("SELECT * FROM athletes", conn)
        hosts_df = pd.read_sql("SELECT * FROM hosts", conn)
        medals_df = pd.read_sql("SELECT * FROM medals", conn)
        results_df = pd.read_sql("SELECT * FROM results", conn)
        
        print(f"✅ Athletes: {len(athletes_df):,} lignes")
        print(f"✅ Hosts: {len(hosts_df):,} lignes")
        print(f"✅ Medals: {len(medals_df):,} lignes")
        print(f"✅ Results: {len(results_df):,} lignes")
        
        conn.close()
        return athletes_df, hosts_df, medals_df, results_df
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        conn.close()
        return None, None, None, None

# Chargement des données
athletes_df, hosts_df, medals_df, results_df = load_olympic_data()

✅ Connexion à Azure MySQL réussie
📊 Chargement des données...
✅ Athletes: 75,904 lignes
✅ Hosts: 53 lignes
✅ Medals: 17,011 lignes
✅ Results: 113,632 lignes
✅ Athletes: 75,904 lignes
✅ Hosts: 53 lignes
✅ Medals: 17,011 lignes
✅ Results: 113,632 lignes


## 2. Exploration et analyse des données

In [ ]:
# Exploration détaillée des données
def explore_dataframe(df, name):
    """Explore un DataFrame en détail"""
    print(f"\n🔍 EXPLORATION - {name.upper()}")
    print("=" * 50)
    
    print(f"📊 Forme: {df.shape}")
    print(f"📋 Colonnes: {list(df.columns)}")
    print(f"🔢 Types de données:")
    print(df.dtypes)
    
    print(f"\n❓ Valeurs manquantes:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Colonne': missing.index,
        'Manquantes': missing.values,
        'Pourcentage': missing_pct.values
    })
    print(missing_df[missing_df['Manquantes'] > 0])
    
    print(f"\n📈 Statistiques descriptives:")
    print(df.describe())
    
    return missing_df

# Exploration de chaque table
if athletes_df is not None:
    athletes_missing = explore_dataframe(athletes_df, "Athletes")
    hosts_missing = explore_dataframe(hosts_df, "Hosts") 
    medals_missing = explore_dataframe(medals_df, "Medals")
    results_missing = explore_dataframe(results_df, "Results")
else:
    print("❌ Données non chargées - vérifiez la connexion")

## 3. Nettoyage et préparation des données

In [ ]:
# Nettoyage et préparation des données
def clean_olympic_data(athletes_df, hosts_df, medals_df, results_df):
    """Nettoie et prépare les données pour l'analyse"""
    print("🧹 NETTOYAGE DES DONNÉES")
    print("=" * 30)
    
    # Copie des DataFrames
    athletes_clean = athletes_df.copy()
    hosts_clean = hosts_df.copy()
    medals_clean = medals_df.copy()
    results_clean = results_df.copy()
    
    # Nettoyage Athletes
    print("👨‍💼 Nettoyage Athletes...")
    athletes_clean = athletes_clean.dropna(subset=['name'])  # Supprime les athlètes sans nom
    athletes_clean['age'] = athletes_clean['age'].fillna(athletes_clean['age'].median())
    print(f"   ✅ {len(athletes_clean):,} athlètes après nettoyage")
    
    # Nettoyage Hosts  
    print("🏟️ Nettoyage Hosts...")
    hosts_clean = hosts_clean.dropna()  # Supprime les lignes avec valeurs manquantes
    print(f"   ✅ {len(hosts_clean):,} hôtes après nettoyage")
    
    # Nettoyage Medals
    print("🥇 Nettoyage Medals...")
    medals_clean = medals_clean.dropna(subset=['athlete_id', 'medal'])
    medals_clean = medals_clean[medals_clean['medal'].isin(['GOLD', 'SILVER', 'BRONZE'])]
    print(f"   ✅ {len(medals_clean):,} médailles après nettoyage")
    
    # Nettoyage Results
    print("🏆 Nettoyage Results...")
    results_clean = results_clean.dropna(subset=['athlete_id', 'ranking'])
    results_clean = results_clean[results_clean['ranking'] > 0]
    print(f"   ✅ {len(results_clean):,} résultats après nettoyage")
    
    return athletes_clean, hosts_clean, medals_clean, results_clean

# Application du nettoyage
if athletes_df is not None:
    athletes_clean, hosts_clean, medals_clean, results_clean = clean_olympic_data(
        athletes_df, hosts_df, medals_df, results_df
    )
    
    # Création d'un dataset unifié pour l'analyse
    print("\n🔗 Création du dataset unifié...")
    
    # Jointure medals avec athletes
    medals_athletes = medals_clean.merge(
        athletes_clean[['id', 'name', 'age', 'nationality']], 
        left_on='athlete_id', 
        right_on='id', 
        how='left'
    )
    
    # Jointure avec hosts pour ajouter les informations de saison
    medals_complete = medals_athletes.merge(
        hosts_clean[['year', 'season', 'country']], 
        on='year', 
        how='left',
        suffixes=('_athlete', '_host')
    )
    
    print(f"✅ Dataset unifié créé: {len(medals_complete):,} lignes")
    print(f"📊 Colonnes: {list(medals_complete.columns)}")
    
else:
    print("❌ Impossible de nettoyer - données non chargées")

## 4. Visualisations interactives avec Plotly

In [ ]:
# Visualisations avec Plotly
def create_olympic_visualizations(medals_complete, hosts_clean, athletes_clean):
    """Crée des visualisations interactives des données olympiques"""
    
    print("📈 CRÉATION DES VISUALISATIONS")
    print("=" * 35)
    
    # 1. Distribution des médailles par type
    print("🥇 Graphique 1: Distribution des médailles...")
    medal_counts = medals_complete['medal'].value_counts()
    
    fig1 = px.pie(
        values=medal_counts.values,
        names=medal_counts.index,
        title="Distribution des médailles olympiques",
        color_discrete_map={'GOLD': '#FFD700', 'SILVER': '#C0C0C0', 'BRONZE': '#CD7F32'}
    )
    fig1.show()
    
    # 2. Médailles par pays hôte (top 10)
    print("🌍 Graphique 2: Top 10 pays par médailles...")
    medals_by_country = medals_complete.groupby('country_host')['medal'].count().sort_values(ascending=False).head(10)
    
    fig2 = px.bar(
        x=medals_by_country.index,
        y=medals_by_country.values,
        title="Top 10 des pays hôtes par nombre de médailles",
        labels={'x': 'Pays hôte', 'y': 'Nombre de médailles'},
        color=medals_by_country.values,
        color_continuous_scale='viridis'
    )
    fig2.show()
    
    # 3. Distribution de l'âge des athlètes
    print("👥 Graphique 3: Distribution des âges...")
    fig3 = px.histogram(
        athletes_clean,
        x='age',
        nbins=30,
        title="Distribution de l'âge des athlètes olympiques",
        labels={'age': 'Âge', 'count': 'Nombre d\'athlètes'}
    )
    fig3.show()
    
    # 4. Médailles par sport (top 15)
    print("🏃‍♂️ Graphique 4: Sports avec le plus de médailles...")
    medals_by_sport = medals_complete['sport'].value_counts().head(15)
    
    fig4 = px.bar(
        x=medals_by_sport.values,
        y=medals_by_sport.index,
        orientation='h',
        title="Top 15 des sports par nombre de médailles",
        labels={'x': 'Nombre de médailles', 'y': 'Sport'}
    )
    fig4.show()
    
    # 5. Évolution des médailles par année
    print("📅 Graphique 5: Évolution temporelle...")
    medals_by_year = medals_complete.groupby(['year', 'medal']).size().reset_index(name='count')
    
    fig5 = px.line(
        medals_by_year,
        x='year',
        y='count',
        color='medal',
        title="Évolution du nombre de médailles par année",
        color_discrete_map={'GOLD': '#FFD700', 'SILVER': '#C0C0C0', 'BRONZE': '#CD7F32'}
    )
    fig5.show()
    
    # 6. Relation âge vs performance (médailles)
    print("📊 Graphique 6: Âge vs Performance...")
    medal_score = {'GOLD': 3, 'SILVER': 2, 'BRONZE': 1}
    medals_complete['medal_score'] = medals_complete['medal'].map(medal_score)
    
    fig6 = px.scatter(
        medals_complete.dropna(subset=['age']),
        x='age',
        y='medal_score',
        color='season',
        title="Relation entre l'âge et la performance (médaille)",
        labels={'age': 'Âge', 'medal_score': 'Score médaille (Or=3, Argent=2, Bronze=1)'}
    )
    fig6.show()
    
    print("✅ Toutes les visualisations créées!")
    
    return fig1, fig2, fig3, fig4, fig5, fig6

# Création des visualisations
if 'medals_complete' in locals():
    visualizations = create_olympic_visualizations(medals_complete, hosts_clean, athletes_clean)
else:
    print("❌ Dataset unifié non disponible - exécutez d'abord les cellules précédentes")

## 5. Ingénierie des caractéristiques (Feature Engineering)

In [ ]:
# Ingénierie des caractéristiques pour le Machine Learning
def create_ml_features(medals_complete, results_clean):
    """Crée et prépare les features pour le machine learning"""
    
    print("⚙️ INGÉNIERIE DES CARACTÉRISTIQUES")
    print("=" * 40)
    
    # Préparation du dataset pour ML
    ml_data = medals_complete.copy()
    
    # 1. Création de nouvelles features
    print("🔧 Création de nouvelles caractéristiques...")
    
    # Catégories d'âge
    ml_data['age_category'] = pd.cut(
        ml_data['age'], 
        bins=[0, 20, 25, 30, 35, 100], 
        labels=['<20', '20-25', '25-30', '30-35', '35+']
    )
    
    # Score de performance par athlète
    athlete_performance = ml_data.groupby('athlete_id').agg({
        'medal_score': ['sum', 'mean', 'count']
    }).round(2)
    athlete_performance.columns = ['total_score', 'avg_score', 'medal_count']
    athlete_performance = athlete_performance.reset_index()
    
    # Ajout au dataset principal
    ml_data = ml_data.merge(athlete_performance, on='athlete_id', how='left')
    
    # 2. Encodage des variables catégorielles
    print("🏷️ Encodage des variables catégorielles...")
    
    # LabelEncoder pour les variables catégorielles
    label_encoders = {}
    categorical_cols = ['sport', 'season', 'country_host', 'medal']
    
    for col in categorical_cols:
        if col in ml_data.columns:
            le = LabelEncoder()
            ml_data[f'{col}_encoded'] = le.fit_transform(ml_data[col].astype(str))
            label_encoders[col] = le
    
    # 3. Features numériques pour ML
    print("🔢 Sélection des features numériques...")
    
    feature_columns = ['age', 'year', 'medal_score', 'total_score', 'avg_score', 'medal_count']
    feature_columns += [f'{col}_encoded' for col in categorical_cols if col != 'medal']
    
    # Création du dataset final pour ML
    ml_features = ml_data[feature_columns + ['medal_encoded']].dropna()
    
    # 4. Normalisation des features
    print("📏 Normalisation des features...")
    
    X = ml_features.drop('medal_encoded', axis=1)
    y = ml_features['medal_encoded']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    
    print(f"✅ Dataset ML créé:")
    print(f"   📊 Forme: {X_scaled_df.shape}")
    print(f"   🎯 Classes cibles: {sorted(y.unique())}")
    print(f"   📋 Features: {list(X.columns)}")
    
    return X_scaled_df, y, scaler, label_encoders, ml_data

# Application de l'ingénierie des features
if 'medals_complete' in locals():
    X_features, y_target, scaler, encoders, ml_dataset = create_ml_features(medals_complete, results_clean)
    
    # Affichage des statistiques
    print(f"\n📈 STATISTIQUES DU DATASET ML")
    print("=" * 35)
    print(f"Nombre d'échantillons: {len(X_features):,}")
    print(f"Nombre de features: {X_features.shape[1]}")
    print(f"Distribution des classes:")
    for medal_code, count in y_target.value_counts().sort_index().items():
        medal_name = {0: 'BRONZE', 1: 'GOLD', 2: 'SILVER'}[medal_code]
        print(f"   {medal_name}: {count:,} ({count/len(y_target)*100:.1f}%)")
        
else:
    print("❌ Dataset unifié non disponible")

## 6. Machine Learning avec Random Forest

In [ ]:
# Préparation des données pour Random Forest
def prepare_ml_data():
    """Prépare les données pour les modèles de Machine Learning"""
    print("Préparation des données pour le Machine Learning...")
    
    # Récupération des données enrichies
    query = """
    SELECT 
        r.athlete_id,
        a.name as athlete_name,
        a.gender,
        r.age,
        r.age_category,
        a.height,
        a.weight,
        r.team,
        r.sport,
        r.event,
        r.medal,
        r.performance_score,
        r.season,
        r.year,
        h.country as host_country,
        h.continent as host_continent
    FROM results r
    JOIN athletes a ON r.athlete_id = a.id
    JOIN hosts h ON r.year = h.year AND r.season = h.season
    WHERE r.age IS NOT NULL 
    AND a.height IS NOT NULL 
    AND a.weight IS NOT NULL
    AND r.medal IS NOT NULL
    """
    
    return pd.read_sql(query, connection)

# Chargement des données
ml_data = prepare_ml_data()
print(f"Données ML: {ml_data.shape}")
print(f"Médailles par type: {ml_data['medal'].value_counts()}")

# Affichage des premières lignes
print("\nPremières lignes des données ML:")
ml_data.head()

In [ ]:
# Préparation des caractéristiques pour Random Forest
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
import seaborn as sns

def prepare_features_rf(data):
    """Prépare les caractéristiques pour Random Forest"""
    df = data.copy()
    
    # Encodage des variables catégorielles
    categorical_features = ['gender', 'team', 'sport', 'event', 'season', 'host_country', 'host_continent', 'age_category']
    
    # Utilisation du LabelEncoder pour chaque variable catégorielle
    encoded_features = {}
    for feature in categorical_features:
        if feature in df.columns:
            le = LabelEncoder()
            df[f'{feature}_encoded'] = le.fit_transform(df[feature].astype(str))
            encoded_features[feature] = le
    
    # Sélection des features numériques et encodées
    feature_columns = ['age', 'height', 'weight', 'performance_score', 'year'] + \
                     [f'{f}_encoded' for f in categorical_features if f in df.columns]
    
    X = df[feature_columns]
    
    return X, encoded_features

# Préparation des données
X, feature_encoders = prepare_features_rf(ml_data)
print(f"Features préparées: {X.shape}")
print(f"Colonnes utilisées: {list(X.columns)}")

# Affichage des statistiques descriptives
print("\nStatistiques des features:")
X.describe()

In [ ]:
# Random Forest Classifier - Prédiction du type de médaille
print("=== RANDOM FOREST CLASSIFIER ===")
print("Prédiction du type de médaille basée sur les caractéristiques de l'athlète")

# Préparation de la variable cible
y_medal = ml_data['medal']

# Division train/test
X_train, X_test, y_train_medal, y_test_medal = train_test_split(
    X, y_medal, test_size=0.2, random_state=42, stratify=y_medal
)

print(f"Données d'entraînement: {X_train.shape}")
print(f"Données de test: {X_test.shape}")
print(f"Distribution des médailles dans l'entraînement:")
print(y_train_medal.value_counts())

# Entraînement du Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

print("\nEntraînement du modèle...")
rf_classifier.fit(X_train, y_train_medal)

# Prédictions
y_pred_medal = rf_classifier.predict(X_test)

# Évaluation
print("\n=== RÉSULTATS DU CLASSIFICATEUR ===")
print(f"Accuracy: {rf_classifier.score(X_test, y_test_medal):.4f}")
print("\nRapport de classification:")
print(classification_report(y_test_medal, y_pred_medal))

In [ ]:
# Matrice de confusion et importance des features
plt.figure(figsize=(15, 5))

# Matrice de confusion
plt.subplot(1, 3, 1)
cm = confusion_matrix(y_test_medal, y_pred_medal)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=rf_classifier.classes_, 
            yticklabels=rf_classifier.classes_)
plt.title('Matrice de Confusion\nPrédiction des Médailles')
plt.xlabel('Prédiction')
plt.ylabel('Réalité')

# Importance des features
plt.subplot(1, 3, 2)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)

plt.barh(range(len(feature_importance)), feature_importance['importance'])
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.title('Importance des Features\nClassification des Médailles')
plt.xlabel('Importance')

# Top 10 des features les plus importantes
plt.subplot(1, 3, 3)
top_features = feature_importance.head(10)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.title('Top 10 Features\nLes Plus Importantes')
plt.xlabel('Importance')

plt.tight_layout()
plt.show()

print("\nTop 10 des features les plus importantes:")
print(feature_importance.head(10))

In [ ]:
# Random Forest Regressor - Prédiction du score de performance
print("\n=== RANDOM FOREST REGRESSOR ===")
print("Prédiction du score de performance basée sur les caractéristiques")

# Préparation de la variable cible pour la régression
y_performance = ml_data['performance_score']

# Division train/test pour la régression
X_train_reg, X_test_reg, y_train_perf, y_test_perf = train_test_split(
    X, y_performance, test_size=0.2, random_state=42
)

# Entraînement du Random Forest Regressor
rf_regressor = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

print("Entraînement du régresseur...")
rf_regressor.fit(X_train_reg, y_train_perf)

# Prédictions
y_pred_perf = rf_regressor.predict(X_test_reg)

# Évaluation
mse = mean_squared_error(y_test_perf, y_pred_perf)
r2 = r2_score(y_test_perf, y_pred_perf)

print(f"\n=== RÉSULTATS DU RÉGRESSEUR ===")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {np.sqrt(mse):.4f}")
print(f"R² Score: {r2:.4f}")

# Validation croisée
cv_scores = cross_val_score(rf_regressor, X_train_reg, y_train_perf, cv=5, scoring='r2')
print(f"\nValidation croisée R² (5-fold): {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 7. Clustering avec K-Means

In [ ]:
# Clustering K-Means des athlètes
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

print("=== CLUSTERING K-MEANS ===")
print("Identification de profils d'athlètes similaires")

# Préparation des données pour le clustering
def prepare_clustering_data():
    """Prépare les données pour le clustering"""
    # Sélection des features numériques pour le clustering
    numeric_features = ['age', 'height', 'weight', 'performance_score', 'year']
    
    # Récupération des données sans valeurs manquantes
    cluster_data = ml_data[numeric_features + ['gender', 'sport', 'medal', 'athlete_name']].copy()
    cluster_data = cluster_data.dropna()
    
    # Standardisation des features numériques
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(cluster_data[numeric_features])
    
    return X_scaled, cluster_data, scaler

X_cluster, cluster_info, scaler_cluster = prepare_clustering_data()
print(f"Données pour clustering: {X_cluster.shape}")

# Détermination du nombre optimal de clusters avec la méthode du coude
inertias = []
silhouette_scores = []
k_range = range(2, 11)

print("\nCalcul du nombre optimal de clusters...")
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_cluster, kmeans.labels_))

# Visualisation de la méthode du coude et du score de silhouette
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Nombre de Clusters (k)')
plt.ylabel('Inertie')
plt.title('Méthode du Coude')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('Nombre de Clusters (k)')
plt.ylabel('Score de Silhouette')
plt.title('Score de Silhouette')
plt.grid(True)

# Choix du nombre optimal de clusters (basé sur le score de silhouette)
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"\nNombre optimal de clusters: {optimal_k}")
print(f"Meilleur score de silhouette: {max(silhouette_scores):.4f}")

# Application du K-Means avec le nombre optimal de clusters
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_cluster)

plt.subplot(1, 3, 3)
plt.scatter(X_cluster[:, 0], X_cluster[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6)
plt.scatter(kmeans_final.cluster_centers_[:, 0], kmeans_final.cluster_centers_[:, 1], 
           c='red', marker='x', s=200, linewidths=3)
plt.xlabel('Âge (standardisé)')
plt.ylabel('Taille (standardisé)')
plt.title(f'Clusters K-Means (k={optimal_k})')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
# Analyse des clusters
cluster_info['cluster'] = cluster_labels

print("=== ANALYSE DES CLUSTERS ===")
print(f"Répartition des athlètes par cluster:")
print(cluster_info['cluster'].value_counts().sort_index())

# Caractéristiques moyennes de chaque cluster
cluster_stats = cluster_info.groupby('cluster').agg({
    'age': ['mean', 'std'],
    'height': ['mean', 'std'],
    'weight': ['mean', 'std'], 
    'performance_score': ['mean', 'std'],
    'year': ['mean', 'min', 'max']
}).round(2)

print("\nCaractéristiques moyennes par cluster:")
print(cluster_stats)

# Analyse par genre et sport
print("\n=== COMPOSITION DES CLUSTERS ===")
for cluster_id in sorted(cluster_info['cluster'].unique()):
    cluster_data = cluster_info[cluster_info['cluster'] == cluster_id]
    print(f"\n--- Cluster {cluster_id} ({len(cluster_data)} athlètes) ---")
    
    # Répartition par genre
    gender_dist = cluster_data['gender'].value_counts()
    print(f"Genre: {dict(gender_dist)}")
    
    # Top 5 sports
    top_sports = cluster_data['sport'].value_counts().head(5)
    print(f"Top 5 sports: {dict(top_sports)}")
    
    # Répartition des médailles
    medal_dist = cluster_data['medal'].value_counts()
    print(f"Médailles: {dict(medal_dist)}")
    
    # Statistiques physiques moyennes
    avg_stats = cluster_data[['age', 'height', 'weight', 'performance_score']].mean()
    print(f"Moyennes: Age={avg_stats['age']:.1f}, Taille={avg_stats['height']:.1f}cm, "
          f"Poids={avg_stats['weight']:.1f}kg, Score={avg_stats['performance_score']:.2f}")

In [ ]:
# Visualisation avancée des clusters
plt.figure(figsize=(20, 15))

# Distribution des clusters par caractéristiques
features_to_plot = ['age', 'height', 'weight', 'performance_score']

for i, feature in enumerate(features_to_plot):
    plt.subplot(3, 4, i+1)
    for cluster_id in sorted(cluster_info['cluster'].unique()):
        cluster_data = cluster_info[cluster_info['cluster'] == cluster_id]
        plt.hist(cluster_data[feature], alpha=0.6, label=f'Cluster {cluster_id}', bins=20)
    plt.xlabel(feature.title())
    plt.ylabel('Fréquence')
    plt.title(f'Distribution de {feature.title()} par Cluster')
    plt.legend()

# Box plots par cluster
for i, feature in enumerate(features_to_plot):
    plt.subplot(3, 4, i+5)
    cluster_info.boxplot(column=feature, by='cluster', ax=plt.gca())
    plt.title(f'Box Plot: {feature.title()} par Cluster')
    plt.suptitle('')

# Heatmap des corrélations par cluster
plt.subplot(3, 4, 9)
correlation_matrix = cluster_info[features_to_plot + ['cluster']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matrice de Corrélation')

# Répartition des médailles par cluster
plt.subplot(3, 4, 10)
medal_cluster = pd.crosstab(cluster_info['cluster'], cluster_info['medal'])
medal_cluster_pct = medal_cluster.div(medal_cluster.sum(axis=1), axis=0)
sns.heatmap(medal_cluster_pct, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Répartition des Médailles\npar Cluster (%)')

# Distribution des sports par cluster (top 10 sports)
plt.subplot(3, 4, 11)
top_sports_overall = cluster_info['sport'].value_counts().head(10).index
sport_cluster = cluster_info[cluster_info['sport'].isin(top_sports_overall)]
sport_counts = pd.crosstab(sport_cluster['cluster'], sport_cluster['sport'])
sport_counts_pct = sport_counts.div(sport_counts.sum(axis=0), axis=1)
sns.heatmap(sport_counts_pct.T, annot=True, fmt='.2f', cmap='Blues')
plt.title('Top 10 Sports par Cluster (%)')
plt.xticks(rotation=0)

# Distribution par genre et cluster
plt.subplot(3, 4, 12)
gender_cluster = pd.crosstab(cluster_info['cluster'], cluster_info['gender'])
gender_cluster.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Répartition Hommes/Femmes\npar Cluster')
plt.xlabel('Cluster')
plt.ylabel('Nombre d\'athlètes')
plt.legend(title='Genre')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

## 8. Évaluation et Comparaison des Modèles

In [ ]:
# Résumé et comparaison des performances des modèles
print("=== RÉSUMÉ DE L'ANALYSE MACHINE LEARNING ===")

# Récapitulatif des modèles
models_summary = {
    'Random Forest Classifier': {
        'Tâche': 'Classification des médailles',
        'Accuracy': rf_classifier.score(X_test, y_test_medal),
        'Features importantes': feature_importance.head(3)['feature'].tolist(),
        'Commentaire': 'Prédiction du type de médaille (Or/Argent/Bronze)'
    },
    'Random Forest Regressor': {
        'Tâche': 'Prédiction du score de performance', 
        'R² Score': r2,
        'RMSE': np.sqrt(mse),
        'CV Score': cv_scores.mean(),
        'Commentaire': 'Prédiction continue du score de performance'
    },
    'K-Means Clustering': {
        'Tâche': 'Segmentation des athlètes',
        'Nombre de clusters': optimal_k,
        'Score de silhouette': max(silhouette_scores),
        'Commentaire': 'Identification de profils d\'athlètes similaires'
    }
}

for model_name, metrics in models_summary.items():
    print(f"\n--- {model_name} ---")
    for metric, value in metrics.items():
        if isinstance(value, float):
            print(f"{metric}: {value:.4f}")
        elif isinstance(value, list):
            print(f"{metric}: {', '.join(value)}")
        else:
            print(f"{metric}: {value}")

# Graphique de comparaison des performances
plt.figure(figsize=(15, 10))

# Performance des modèles
plt.subplot(2, 3, 1)
models = ['RF Classifier', 'RF Regressor', 'K-Means']
scores = [
    rf_classifier.score(X_test, y_test_medal),
    r2,
    max(silhouette_scores)
]
colors = ['skyblue', 'lightgreen', 'lightcoral']
bars = plt.bar(models, scores, color=colors)
plt.title('Scores de Performance des Modèles')
plt.ylabel('Score')
plt.ylim(0, 1)
for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom')

# Distribution des prédictions vs réalité (Classifier)
plt.subplot(2, 3, 2)
pred_vs_real = pd.DataFrame({'Réel': y_test_medal, 'Prédit': y_pred_medal})
confusion_data = pd.crosstab(pred_vs_real['Réel'], pred_vs_real['Prédit'])
sns.heatmap(confusion_data, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de Confusion\nClassificateur')

# Scatter plot prédictions vs réalité (Regressor)
plt.subplot(2, 3, 3)
plt.scatter(y_test_perf, y_pred_perf, alpha=0.6)
plt.plot([y_test_perf.min(), y_test_perf.max()], [y_test_perf.min(), y_test_perf.max()], 'r--', lw=2)
plt.xlabel('Score Réel')
plt.ylabel('Score Prédit')
plt.title('Prédictions vs Réalité\nRégresseur')

# Importance des features (top 10)
plt.subplot(2, 3, 4)
top_10_features = feature_importance.head(10)
plt.barh(range(len(top_10_features)), top_10_features['importance'])
plt.yticks(range(len(top_10_features)), top_10_features['feature'])
plt.title('Top 10 Features Importantes')
plt.xlabel('Importance')

# Distribution des clusters
plt.subplot(2, 3, 5)
cluster_counts = cluster_info['cluster'].value_counts().sort_index()
plt.pie(cluster_counts.values, labels=[f'Cluster {i}' for i in cluster_counts.index], 
        autopct='%1.1f%%', startangle=90)
plt.title('Répartition des Athlètes\npar Cluster')

# Évolution temporelle par cluster
plt.subplot(2, 3, 6)
for cluster_id in sorted(cluster_info['cluster'].unique()):
    cluster_data = cluster_info[cluster_info['cluster'] == cluster_id]
    yearly_counts = cluster_data.groupby('year').size()
    plt.plot(yearly_counts.index, yearly_counts.values, marker='o', label=f'Cluster {cluster_id}')
plt.xlabel('Année')
plt.ylabel('Nombre d\'athlètes')
plt.title('Évolution Temporelle\ndes Clusters')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 9. Conclusions et Insights

In [ ]:
# Insights finaux et recommandations
print("=== INSIGHTS CLÉS DE L'ANALYSE ===")

insights = [
    "🏅 CLASSIFICATION DES MÉDAILLES:",
    f"- Le modèle Random Forest atteint une accuracy de {rf_classifier.score(X_test, y_test_medal):.3f}",
    f"- Les features les plus importantes sont: {', '.join(feature_importance.head(3)['feature'].tolist())}",
    "- Cela suggère que certaines caractéristiques sont prédictives du succès olympique",
    "",
    "📊 PRÉDICTION DE PERFORMANCE:",
    f"- Le régresseur Random Forest obtient un R² de {r2:.3f}",
    f"- RMSE de {np.sqrt(mse):.3f} sur l'échelle de score de performance",
    "- Validation croisée confirme la robustesse du modèle",
    "",
    "👥 SEGMENTATION DES ATHLÈTES:",
    f"- {optimal_k} clusters identifiés avec un score de silhouette de {max(silhouette_scores):.3f}",
    "- Chaque cluster représente un profil d'athlète distinct",
    "- Permet de comprendre la diversité des profils olympiques",
    "",
    "🔍 RECOMMANDATIONS:",
    "1. Utiliser les features importantes pour le recrutement sportif",
    "2. Adapter l'entraînement selon le profil de cluster de l'athlète",
    "3. Prédire les performances futures pour la planification olympique",
    "4. Analyser l'évolution des profils au fil du temps",
    "",
    "💡 EXTENSIONS POSSIBLES:",
    "- Modèles plus complexes (XGBoost, réseaux de neurones)",
    "- Analyse temporelle des tendances",
    "- Prédiction par sport spécifique",
    "- Analyse de l'impact des pays hôtes",
    "- Clustering hiérarchique pour une segmentation plus fine"
]

for insight in insights:
    print(insight)

# Sauvegarde des résultats pour utilisation future
results_summary = {
    'rf_classifier_accuracy': rf_classifier.score(X_test, y_test_medal),
    'rf_regressor_r2': r2,
    'optimal_clusters': optimal_k,
    'silhouette_score': max(silhouette_scores),
    'top_features': feature_importance.head(5)['feature'].tolist(),
    'total_athletes_analyzed': len(ml_data),
    'clusters_distribution': cluster_info['cluster'].value_counts().to_dict()
}

print(f"\n=== RÉSULTATS SAUVEGARDÉS ===")
print("Les métriques clés ont été calculées et sont disponibles pour analyse future.")
print(f"Nombre total d'athlètes analysés: {results_summary['total_athletes_analyzed']:,}")

# Affichage final des métriques de performance
print(f"\n=== MÉTRIQUES FINALES ===")
print(f"🎯 Classification Accuracy: {results_summary['rf_classifier_accuracy']:.3f}")
print(f"📈 Regression R²: {results_summary['rf_regressor_r2']:.3f}")
print(f"🔄 Clustering Silhouette: {results_summary['silhouette_score']:.3f}")
print(f"📊 Nombre de clusters: {results_summary['optimal_clusters']}")

print("\n✅ ANALYSE COMPLÈTE TERMINÉE ✅")
print("Tous les modèles ML ont été entraînés et évalués avec succès!")